In [6]:
import pandas as pd
import re
import string
from colorama import Fore
import kagglehub

In [7]:
# Download latest version
path = kagglehub.dataset_download("dannyleeakira/dataset-for-academic-novelty-insight-survey")

print("Path to dataset files:", path)

100%|██████████| 41.0M/41.0M [00:02<00:00, 20.5MB/s]

Extracting files...


Path to dataset files: /Users/rikenkon/.cache/kagglehub/datasets/dannyleeakira/dataset-for-academic-novelty-insight-survey/versions/7


In [10]:
#HotpotQa paper_content from S2orc,Source in https://doi.org/10.34740/KAGGLE/DS/4330260
df=pd.read_csv(f'{path}/Hotpotqa_paper_content.csv',index_col=0)

,corpusid,externalids,content,updated,ref_paper_id
0,263908862,"{'arxiv': '2310.08541', 'mag': None, 'acl': No...",{'source': {'pdfurls': ['https://export.arxiv....,NaN,{}
1,248665596,"{'arxiv': None, 'mag': None, 'acl': None, 'pub...",{'source': {'pdfurls': ['https://arxiv.org/pdf...,NaN,{220302524: '[30]'}
2,6729528,"{'arxiv': '1406.0930', 'mag': '2296706549', 'a...",{'source': {'pdfurls': ['https://arxiv.org/pdf...,NaN,{}
3,264490922,"{'arxiv': None, 'mag': None, 'acl': None, 'pub...",{'source': {'pdfurls': ['https://export.arxiv....,NaN,"{123758373: 'Chen and Durrett, 2019;', 2466523..."
4,14155615,"{'arxiv': None, 'mag': '2065435633', 'acl': No...","{'source': {'pdfurls': None, 'pdfsha': '8cd7af...",NaN,NaN
...,...,...,...,...,...
1534,236459873,"{'arxiv': None, 'mag': None, 'acl': '2021.acl-...",{'source': {'pdfurls': ['https://aclanthology....,NaN,"{218487313: 'Yadav et al., 2020', 52822214: 'Y..."
1535,2724512,"{'arxiv': None, 'mag': '2021858754', 'acl': No...","{'source': {'pdfurls': None, 'pdfsha': 'fb186d...",NaN,{9060527: '[41]'}
1536,202565945,"{'arxiv': '1909.05803', 'mag': '2972378157', '...",{'source': {'pdfurls': ['https://www.aclweb.or...,NaN,"{139103297: 'Chen and Durrett, 2019;', 1899278..."
1537,248476403,"{'arxiv': None, 'mag': None, 'acl': None, 'pub...",{'source': {'pdfurls': ['https://arxiv.org/pdf...,NaN,"{230799347: 'Geva et al., 2021', 225075843: 'L..."


In [ ]:
#Get offset of text
def get_offset(dict,i):
    start=dic[i]['start']
    end=dic[i]['end']
    return start,end

In [ ]:
#Extract text of paper-title by using annotation and offset  
title_list=[]
for i in range(len(df)):
    a=eval(df['content'][i])
    if a['annotations']['title']==None:
        title_list.append('None')
    else:
        dic=eval(a['annotations']['title'])
        start,end = get_offset(dic,0)
        title_list.append(a['text'][start:end])

In [ ]:
# Insert paper-title column
df.insert(1,'title',title_list)
df

In [ ]:
#Extract text from Section 'limitation','Disscussion','Conclusion'
columns = ['corpusid','paper-title','section-title','Target','Content','ref_paper_id','pdfurl']
d_sec=pd.DataFrame(columns=columns)

strings=''
for i in range(len(df)):
    a=eval(df['content'][i])
    position=[]
    text_end=len(a['text'])-1

    #URL
    try:
        url=a['source']['pdfurls'][0]
    except:
        url='NaN'
    
    #section
    try:
        dic=eval(a['annotations']['sectionheader'])
    except:
        continue

    for j in range(len(dic)):
        if 'attributes' not in list(dic[j].keys()): # section_num strings in section
            continue
        
        start,end = get_offset(dic,j)
        try:
            start_next,end_next=get_offset(dic,j+1)
        except:
            print('bottom')
        # print('start:',start)
        sec_num=dic[j]['attributes']['n']
        section_title = a['text'][start:end].lower()
            
        if 'disscuss' in section_title and 'conclusion' not in section_title:
            Total=a['text'][end:start_next].lower()

            print(Fore.GREEN +'disscussion:',Total.replace('\n',''))
            strings=Total.replace('\n','')
            # add-data
            d_sec_append=pd.DataFrame(data=[[df['corpusid'][i],df['title'][i],section_title,'Insight-tree',strings,df['ref_paper_id'][i],url]], columns=columns)
            d_sec=pd.concat([d_sec, d_sec_append], ignore_index=True, axis=0)

        if 'limitation' in section_title and 'conclusion' not in section_title:
            Total=a['text'][end:start_next].lower()

            print(Fore.GREEN +'limitation:',Total.replace('\n',''))
            strings=Total.replace('\n','')
            # add-data
            d_sec_append=pd.DataFrame(data=[[df['corpusid'][i],df['title'][i],section_title,'Insight-tree',strings,df['ref_paper_id'][i],url]], columns=columns)
            d_sec=pd.concat([d_sec, d_sec_append], ignore_index=True, axis=0)

        
        if 'conclusion' in section_title:
            for p in range(len(eval(a['annotations']['paragraph']))):
                ref_flag=eval(a['annotations']['paragraph'])[p]['end']
                if ref_flag > end:
                    break
            
            Total=a['text'][end:ref_flag].lower()

            print(Fore.GREEN +'conclusion:',Total.replace('\n',''))
            strings=Total.replace('\n','')
            # add-data
            d_sec_append=pd.DataFrame(data=[[df['corpusid'][i],df['title'][i],section_title,'Insight-tree',strings,df['ref_paper_id'][i],url]], columns=columns)
            d_sec=pd.concat([d_sec, d_sec_append], ignore_index=True, axis=0)
        
        print(Fore.MAGENTA + 'Section_title:',sec_num + ' ',a['text'][start:end])
        end_pos=end
        position.append(end_pos)
        
    print('\n')
    print('#########')

In [ ]:
Insight_dataset=d_sec[d_sec['Target']=='Insight-tree']
Insight_dataset

In [ ]:
Insight_dataset=Insight_dataset.reset_index(drop=True)

In [ ]:
#Delete rows with lack of information
emp=[]
for i in range(len(Insight_dataset)):
    text=Insight_dataset['Content'][i]
    if len(text)<=10:
        emp.append(i)

Insight_dataset.drop(index=emp, inplace=True)

Insight_dataset=Insight_dataset.reset_index(drop=True)

In [ ]:
#Drop_duplicates
Insight_dataset=Insight_dataset.drop_duplicates("Content")
Insight_dataset=Insight_dataset.reset_index(drop=True)

In [ ]:
Insight_dataset.insert(7,'ReSolved','None')
Insight_dataset.insert(8,'Finding','None')
Insight_dataset=Insight_dataset.reset_index(drop=True)

In [ ]:
Insight_dataset.to_csv('./Data/Insight_dataset.csv')

# Sentence segmentation

In [ ]:
import spacy
import pandas as pd

In [ ]:
# Install en_core_scibert for sentence segmentation
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_sci_lg-0.5.4.tar.gz

In [ ]:
#load en_core_sci_lg
nlp = spacy.load('en_core_sci_lg')

In [ ]:
columns = ['corpusid','paper-title','section-title','Target','Sentence','pdf-url']
d_sentences=pd.DataFrame(columns=columns)

for i in range(len(Insight_dataset)):
    doc=nlp(Insight_dataset['Content'][i])
    for sentence in doc.sents:
        # print(sentence)
        # add-data
        text=str(sentence)
        d_sentences_append=pd.DataFrame(data=[[Insight_dataset['corpusid'][i],Insight_dataset['paper-title'][i],Insight_dataset['section-title'][i],'Insight-tree',text,Insight_dataset['pdfurl'][i]]], columns=columns)
        d_sentences=pd.concat([d_sentences, d_sentences_append], ignore_index=True, axis=0)

In [ ]:
d_sentences

In [ ]:
d_sentences.to_csv('./Data/Insight_sentences.csv')